In [ ]:
import keras
import numpy as np
import pandas as pd
from time import time
#from utils import vis
from keras.models import Sequential
from keras.layers import LSTM, Dense
#from keras.utils import to_categorical
from keras.utils.np_utils import to_categorical
from keras.callbacks import TensorBoard
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.client import device_lib
from sklearn.model_selection import train_test_split

In [ ]:
# get data
data = pd.read_csv('Level_2_ZyMoC1_EsCoC2_PsPuC3_1188.csv') 

# split data
X=data.iloc[:,1:151]
y=data.iloc[:,151].astype('category')

print(X)
print(y)

          Xc1.A      Xc1.R  ...  Grantham.Xd.29  Grantham.Xd.30
0     14.767060  10.829177  ...        0.032505        0.032749
1     11.728905   8.138424  ...        0.031626        0.032946
2      8.319776   2.773259  ...        0.031596        0.031098
3     10.255686   3.778411  ...        0.031188        0.031233
4      6.745884   2.698354  ...        0.031939        0.030060
...         ...        ...  ...             ...             ...
1183   3.434023   2.113245  ...        0.029650        0.032397
1184   8.597566   6.513308  ...        0.032343        0.028946
1185   8.597410   1.719482  ...        0.033358        0.028789
1186   5.471856   0.521129  ...        0.032696        0.027279
1187   5.471856   0.521129  ...        0.032696        0.027279

[1188 rows x 150 columns]
0       1
1       1
2       1
3       1
4       1
       ..
1183    3
1184    3
1185    3
1186    3
1187    3
Name: Class, Length: 1188, dtype: category
Categories (3, int64): [1, 2, 3]


In [ ]:
# init
n_epochs = 1000
n_comps = X.shape[1]

In [ ]:
# encode the classification labels
#le = LabelEncoder()
#yy = to_categorical(le.fit_transform(y))

from collections import Counter
from imblearn.over_sampling import SMOTE 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


lda = LinearDiscriminantAnalysis(n_components=10)
X_ld = lda.fit(X, y).transform(X)

le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

sm = SMOTE(random_state=45)
x_b, y_b = sm.fit_resample(X_ld, yy)

#print('Resampled dataset shape %s' % Counter(Y_res))

# split the dataset
x_train, x_test, y_train, y_test = train_test_split(x_b, y_b, test_size=0.20,
                                                    random_state=True,
                                                    shuffle=True)

# exapand dimensions
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:463: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(150, 3 - 1) = 2 components.
  ChangedBehaviorWarn

In [ ]:
# define model
model = Sequential()
model.add(LSTM(units=n_comps, return_sequences=True,
               input_shape=(x_train.shape[1], 1),
               dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(128, activation='tanh', return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(64, activation='tanh',  dropout=0.2, recurrent_dropout=0.2))



model.add(Dense(3 ,activation='softmax'))

# print model architecture summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 2, 150)            91200     
_________________________________________________________________
lstm_1 (LSTM)                (None, 2, 128)            142848    
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 3)                 195       
Total params: 283,651
Trainable params: 283,651
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, epochs=n_epochs, batch_size=32,
                    validation_data=(x_test, y_test))

Epoch 1/20
41/41 [==============================] - 33s 83ms/step - loss: 0.6561 - accuracy: 0.3368 - val_loss: 0.6257 - val_accuracy: 0.5360
Epoch 2/20
41/41 [==============================] - 1s 19ms/step - loss: 0.6167 - accuracy: 0.4315 - val_loss: 0.5474 - val_accuracy: 0.6148
Epoch 3/20
41/41 [==============================] - 1s 19ms/step - loss: 0.5603 - accuracy: 0.5132 - val_loss: 0.5325 - val_accuracy: 0.5940
Epoch 4/20
41/41 [==============================] - 1s 19ms/step - loss: 0.5606 - accuracy: 0.5559 - val_loss: 0.4980 - val_accuracy: 0.7077
Epoch 5/20
41/41 [==============================] - 1s 19ms/step - loss: 0.5267 - accuracy: 0.6281 - val_loss: 0.4563 - val_accuracy: 0.7216
Epoch 6/20
41/41 [==============================] - 1s 19ms/step - loss: 0.4840 - accuracy: 0.6425 - val_loss: 0.4189 - val_accuracy: 0.7471
Epoch 7/20
41/41 [==============================] - 1s 19ms/step - loss: 0.4763 - accuracy: 0.6497 - val_loss: 0.3994 - val_accuracy: 0.7401
Epoch 8/20
4

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
#from sklearn.metrics import roc_auc_score
import itertools

train_pred = model.predict(x_train)
test_pred = model.predict(x_test)
print("train-acc = " + str(accuracy_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-acc = " + str(accuracy_score(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
print("train-CFR = " + str(classification_report(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))
print("test-CFR = " + str(classification_report(np.argmax(y_test, axis=1), np.argmax(test_pred, axis=1))))
#print((roc_auc_score(np.argmax(y_train, axis=1), np.argmax(train_pred, axis=1))))

train-acc = 0.7242447714949651
test-acc = 0.7563805104408353
train-CFR =               precision    recall  f1-score   support

           0       0.70      0.81      0.75       426
           1       0.79      0.57      0.66       438
           2       0.71      0.80      0.75       427

    accuracy                           0.72      1291
   macro avg       0.73      0.73      0.72      1291
weighted avg       0.73      0.72      0.72      1291

test-CFR =               precision    recall  f1-score   support

           0       0.78      0.86      0.82       148
           1       0.76      0.60      0.67       136
           2       0.73      0.80      0.76       147

    accuracy                           0.76       431
   macro avg       0.76      0.75      0.75       431
weighted avg       0.76      0.76      0.75       431

